<a href="https://colab.research.google.com/github/FeelOneE/Kaggle_study/blob/main/Spooky_Author_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spooky Author Identification
# https://www.kaggle.com/c/spooky-author-identification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [37]:
% cd /content/drive/MyDrive/kaggle
% ls -al

/content/drive/MyDrive/kaggle
total 6396
-rw------- 1 root root      65 Jan 13 09:17 kaggle.json
-rw------- 1 root root   30082 Jan 13 09:23 sample_submission.zip
-rw------- 1 root root 1351241 Oct 25  2017 test.csv
-rw------- 1 root root  551200 Jan 13 09:23 test.zip
-rw------- 1 root root 3295644 Oct 25  2017 train.csv
-rw------- 1 root root 1319237 Jan 13 09:23 train.zip


# 캐글 데이터 세트 로드

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json
# 내가 참가한 대회 리스트 확인
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         76           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        107           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        909           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        177           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1896           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      13380            True  
house-pr

In [ ]:
!kaggle competitions download -c spooky-author-identification

  0% 0.00/29.4k [00:00<?, ?B/s]
100% 29.4k/29.4k [00:00<00:00, 3.74MB/s]
  0% 0.00/538k [00:00<?, ?B/s]
100% 538k/538k [00:00<00:00, 29.8MB/s]
  0% 0.00/1.26M [00:00<?, ?B/s]
100% 1.26M/1.26M [00:00<00:00, 83.5MB/s]


In [ ]:
! unzip test.zip

Archive:  test.zip
  inflating: test.csv                


In [ ]:
! unzip train.zip

Archive:  train.zip
  inflating: train.csv               


In [38]:
! ls -al

total 6396
-rw------- 1 root root      65 Jan 13 09:17 kaggle.json
-rw------- 1 root root   30082 Jan 13 09:23 sample_submission.zip
-rw------- 1 root root 1351241 Oct 25  2017 test.csv
-rw------- 1 root root  551200 Jan 13 09:23 test.zip
-rw------- 1 root root 3295644 Oct 25  2017 train.csv
-rw------- 1 root root 1319237 Jan 13 09:23 train.zip


In [39]:
import pandas as pd

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [40]:
df_test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


#토크나이저 로딩

1. 최대 길이 초과 제거
2. 고정 길이 패딩
3. 어텐션 마스크 만들기


In [ ]:
!pip install transformers

In [42]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# BERT 모델 로드

In [43]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback

------------------------------- 참고 내용 -----------------------------------------
https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b

In [44]:
# Read data
data = pd.read_csv("train.csv")
data.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [45]:
# ----- 1. Preprocess data -----#
# Preprocess data
X = list(data["text"])
y = list(data["author"])

BATCH_SIZE = 16

# x_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=512, add_special_tokens = True,return_attention_mask=True)
# x_test_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=512)

In [46]:
# 라벨(y 데이터) 원핫 인코딩 하기
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
np_y = le.transform(y)
np_y[:30]

array([0, 1, 0, 2, 1, 2, 0, 0, 0, 2, 2, 0, 1, 1, 0, 2, 0, 2, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1])

In [47]:
x_train, x_test, y_train, y_test = train_test_split(X, np_y, test_size=0.2)

In [48]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(tokenizer(x_train, padding=True, truncation=True, max_length=512, add_special_tokens = True,return_attention_mask=True, return_tensors="tf")),  # Convert BatchEncoding instance to dictionary
    y_train
)).shuffle(1000).batch(BATCH_SIZE).prefetch(1)

validation_dataset = tf.data.Dataset.from_tensor_slices((
    dict(tokenizer(x_test, padding=True, truncation=True, max_length=512, return_tensors="tf")),
    y_test
)).batch(BATCH_SIZE).prefetch(1)

In [75]:
validation_dataset = tf.data.Dataset.from_tensor_slices((
    dict(tokenizer(x_test, padding=True, truncation=True, max_length=512, return_tensors="tf")),
    y_test
))

In [49]:
train_dataset

<PrefetchDataset shapes: ({input_ids: (None, 512), token_type_ids: (None, 512), attention_mask: (None, 512)}, (None,)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>

In [77]:
# 데이터 셋 확인하기
cnt = 0
for i in validation_dataset:
  print(i)
  print('-'*20)
  print(i[0]['input_ids'][0])
  #print(i[0]['token_type_ids'])
  print(i[0]['attention_mask'][0])
  #print('-'*20)
  #print(i[1])
  cnt += 1
  if cnt > 0:
    break

({'input_ids': <tf.Tensor: shape=(211,), dtype=int32, numpy=
array([  101,  1045,  2031,  8729,  2010,  8087,  2075,  1996,  4297,
        2239,  8159, 13684,  2015,  1998,  3383, 16796,  1997,  2061,
        2146,  1037,  4990,  1010,  2664,  2129,  2411,  2031,  1045,
       18991,  2025,  2108,  2583,  2000,  4685,  2009,  2870,  1045,
        3275,  2000,  2870,  2008,  1996,  4708,  1997,  7052,  2006,
        2115,  5305,  8270,  2038, 16475, 16116,  2006,  2070, 22146,
        2214,  6821,  1010,  2040,  2071,  2196,  3984,  2115,  8996,
        4496,  2704,  2000,  2068,  2007,  1996,  2729,  1998, 12242,
        1997,  2115,  3532,  5542,  1012,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,  

---- 모델링 ----

In [51]:
from transformers import TFBertForSequenceClassification, AdamW, BertConfig


# 참고 자료
#
## https://stackoverflow.com/questions/60463829/training-tfbertforsequenceclassification-with-custom-x-and-y-data
#
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

#model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# recommended learning rate for Adam 5e-5, 3e-5, 2e-5

learning_rate = 2e-5

# we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
number_of_epochs = 1

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________


In [53]:
# 트랜스포머 가중치 얼리기
model.layers[0].trainable = False

In [54]:
model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109,484,547
Trainable params: 2,307
Non-trainable params: 109,482,240
_________________________________________________________________


In [80]:
model.fit(x=train_dataset,
          y=None, 
          validation_data=validation_dataset,
          epochs=number_of_epochs,
          batch_size=BATCH_SIZE ,
          )

  4/979 [..............................] - ETA: 3:04:42 - loss: 0.3228 - accuracy: 0.8906

KeyboardInterrupt: ignored

In [94]:
model.predict(
    dict(tokenizer(x_train[1101:1105], padding=True, truncation=True, max_length=512, add_special_tokens = True,return_attention_mask=True, return_tensors="tf")) 
     )

TFSequenceClassifierOutput([('logits',
                             array([[-0.01045346,  2.2298448 , -2.5355706 ],
                                    [-1.4468687 , -1.730755  ,  4.1463423 ],
                                    [-0.86544913,  0.42552808,  0.43961877],
                                    [-1.7794688 , -0.83263224,  3.5393267 ]], dtype=float32))])

In [95]:
y_train[1101:1105]

array([1, 2, 1, 2])

In [82]:
tf.saved_model.save(model, "./model")

INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: ./model/assets


In [ ]:
loaded = tf.saved_model.load("./model")